In [19]:
import json
import boto3
import math
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [4]:
session = boto3.Session()

bedrock = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1',
    endpoint_url="https://bedrock-runtime.us-east-1.amazonaws.com"
)

In [5]:
def get_embedding(text):
    body = json.dumps({"inputText": text})
    model_d = 'amazon.titan-embed-text-v1'
    mime_type = 'application/json'
    response = bedrock.invoke_model(body=body, modelId=model_d, accept=mime_type, contentType=mime_type)
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    return embedding

In [8]:
embedding_result = get_embedding('저는 배가 고파요')

In [9]:
len(embedding_result)

1536

In [12]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [13]:
df['embedding'] = df.apply(lambda row: get_embedding(
        row.text,
    ), axis=1)

In [14]:
df

,text,embedding
0,저는 배가 고파요,"[-0.6953125, -0.14160156, -0.5625, 0.20800781,..."
1,저기 배가 지나가네요,"[0.15136719, -0.11669922, -0.44140625, 0.14062..."
2,굶어서 허기가 지네요,"[-0.15625, -0.36328125, -0.58203125, 0.4121093..."
3,허기 워기라는 게임이 있는데 즐거워,"[0.82421875, -0.6484375, -0.038085938, 0.21289..."
4,스팀에서 재밌는 거 해야지,"[0.27929688, -0.52734375, -0.0008430481, 0.248..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[0.07421875, -0.37890625, 0.14257812, 0.141601..."


In [20]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(
        query,
    )
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x),
                                                            np.array(query_embedding)))
    results_co = df.sort_values("similarity",
                                ascending=False,
                                ignore_index=True).iloc[:3, :]
    return results_co.head(3)

In [21]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가나네')
sim_result

,text,embedding,similarity
0,저는 배가 고파요,"[-0.6953125, -0.14160156, -0.5625, 0.20800781,...",0.647040
1,굶어서 허기가 지네요,"[-0.15625, -0.36328125, -0.58203125, 0.4121093...",0.622445
2,스팀에어프라이어로 연어구이 해먹을거야,"[0.07421875, -0.37890625, 0.14257812, 0.141601...",0.591753
